In [ ]:
import specpy as sp
import numpy as np
from itertools import count
from pprint import pprint
import json
from queue import PriorityQueue

In [ ]:
im = sp.Imspector()
im.version()

In [ ]:
class AcquisitionPriorityQueue(PriorityQueue):
    def __init__(self):
        PriorityQueue.__init__(self)
        self.ctr = count()
    def put(self, item, prio):
        PriorityQueue.put(self, (prio, next(self.ctr), item))
    def get(self, *args, **kwargs):
        lvl, _, item = PriorityQueue.get(self, *args, **kwargs)
        return (lvl, item)
        
class _pipeline_level:
    def __init__(self, parent, name):
        self.parent = parent
        self.name = name
    
    def __eq__(self, other):
        return self.name == other.name
    
    def __le__(self, other):
        return self.parent.levels.index(self) <= self.parent.levels.index(other)
    
    def __lt__(self, other):
        return self.parent.levels.index(self) < self.parent.levels.index(other)
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name
        

class PipelineLevels:
    levels = []
    def __init__(self, *args):
        for arg in reversed(args):
            lvl = _pipeline_level(self, arg)
            self.levels.append(lvl)
            setattr(self, arg, lvl)

            
pl = PipelineLevels('overview', 'detail', 'minfield')
        
pq = AcquisitionPriorityQueue()

def done_and_enqueue_sted(no, nsteds, qu, pip):
    print('ov {} done'.format(no))
    for _ in range(nsteds):
        qu.put( lambda : print('sted done'), pip.sted)

pq.put( lambda : done_and_enqueue_sted(0, 5, pq, pl) , pl.overview)
pq.put( lambda : done_and_enqueue_sted(1, 5, pq, pl) , pl.overview)

while not pq.empty():
    lvl, fun = pq.get()
    fun()

print('Done')

In [ ]:
it = count()


ms = im.create_measurement()
ms.set_parameters('', params)

im.connect_begin(lambda : next(it), 1)
im.run(ms)

params = ms.parameters('')
js = json.dumps(ms.parameters(''), indent=2)

im.close(ms)
print(next(it))

print(js)
justoix = dict([(k,v) for k,v in params.items() if (k == 'OlympusIX')])

#pprint(im.parameters(''))